## Advanced Features

This notebook serves as an introduction to some of the advanced features of GraPE, such as a RayTune wrapper and other hybrid models.

### Ray Tune

Ray (https://github.com/ray-project/ray) and specifically Ray Tune is a package offers streamlined hyperparameter optimization. We include a simple wrapper in our package that can aide in fast implementation.

For hyperparameter optimization, we need three things:
* a parameter search space
* an objective function which contains training and validation
* a ray tuner

We begin with loading our data and initializing model:

In [1]:
from torch import nn
from grape.datasets import BradleyDoublePlus
from grape.models import MGConv, SimpleGNN

# loading our data and splitting it
data = BradleyDoublePlus(split_type='random')
train_data, val_data, test_data = data.train, data.val, data.test

node_hidden_dim = 128
batch_size = 32

# Weave Message Passing NN
model_message = MGConv(edge_hidden_dim=128, node_hidden_dim=node_hidden_dim, node_in_dim=data.num_node_features, edge_in_dim=data.num_edge_features)

# output MLP
out_model = nn.Sequential(
    nn.Linear(node_hidden_dim, 128),
    nn.ReLU(),
    nn.Linear(128, 1)
)


model = lambda : SimpleGNN(model_message=model_message,
                  out_model=out_model)

In the following, we will be optimizing the parameters of our Adam optimizer. To do so, we use the adam optimizer objective from the toolbox as well as a default search space:

In [2]:
from grape.optim import adam_objective, adam_default_search_space
search_space = adam_default_search_space()
objective = adam_objective

Now we need to initialize our Ray Tuner. The objective function usually needs to be self-contained, so loading, training and testing in one function, as we are re-initializing our model during optimization. Our pre-defined objectives take care of that, however.

In [3]:
from grape.utils import RayTuner
from ray.tune.search.bayesopt import BayesOptSearch

tuner = RayTuner(search_space=search_space, objective=objective, 
                 train_loader=train_data, val_loader=val_data, model=model,
                 search_algo=BayesOptSearch,train_iterations=150)

We are using the BayesOpt search algorithm (https://github.com/bayesian-optimization/BayesianOptimization), but any search algorithm compatible with Ray Tune can be used (https://docs.ray.io/en/latest/tune/api/suggestion.html):

In [5]:
tuner.fit()

2024-04-17 13:34:58,021	INFO tune.py:1048 -- Total run time: 108.87 seconds (108.56 seconds for the tuning loop).


Best config is:  {'lr': 0.003807947176588889, 'weight_decay': 0.00951207163345817}


In [6]:
print('Best config is: ', tuner.results.get_best_result().config)

Best config is:  {'lr': 0.003807947176588889, 'weight_decay': 0.00951207163345817}
